# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

In [1]:
# first observation,nice little detail on the "her" part ,so i gues we have to assume that the boss is Queen Erin.

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
orders = pd.read_csv("Orders.csv")
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# Sub Problem 1: How to aggregate the amount_spent for unique customers?
customer_total_spent = orders.groupby("CustomerID")["amount_spent"].sum().reset_index()
customer_total_spent.columns = ["CustomerID","TotalSpent"]
customer_total_spent
# so we pretty much just group the customers to see how much they have spent

,CustomerID,TotalSpent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [5]:
# This values will be a bit invented or in my opinoon VIp should be top 5% of the total amount spent ,preferded customers next 15%

vip_threshold = customer_total_spent["TotalSpent"].quantile(0.95)
preferred_threshold = customer_total_spent["TotalSpent"].quantile(0.80)


In [6]:
# Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?
#total spent greater than vip_threshold should be labeled as vip and totalspent between preffered_threshold and vip_threshold will be labeles as preffered
def label_customer(row):
    if row["TotalSpent"] > vip_threshold:
        return "VIP"
    elif row["TotalSpent"] > preferred_threshold: # because we first check if its a vip,this will only take the ones that are not Vip but higher than the preffered threshold
        return "Preferred"
    else:
        return "Regular"
customer_total_spent["CustomerLabel"] = customer_total_spent.apply(label_customer, axis = 1)
customer_total_spent

,CustomerID,TotalSpent,CustomerLabel
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Regular
3,12349,1757.55,Regular
4,12350,334.40,Regular
...,...,...,...
4334,18280,180.60,Regular
4335,18281,80.82,Regular
4336,18282,178.05,Regular
4337,18283,2094.88,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [7]:
# first merge the new df with old df
merged_orders = orders.merge(customer_total_spent[["CustomerID","CustomerLabel"]], on="CustomerID", how="left")
merged_orders
# the how='left' ensures that all rows in the original dataframe are retained, and the labels are matched where they exist

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,CustomerLabel
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,Regular
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,Regular
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Regular
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Regular


In [8]:
# filter the dataframe to have only rows with vip customers
vip_customers = merged_orders[merged_orders['CustomerLabel'] == 'VIP']
vip_customers

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,CustomerLabel
26,26,536370,22728,2010,12,3,8,alarm clock bakelike pink,24,2010-12-01 08:45:00,3.75,12583,France,90.0,VIP
27,27,536370,22727,2010,12,3,8,alarm clock bakelike red,24,2010-12-01 08:45:00,3.75,12583,France,90.0,VIP
28,28,536370,22726,2010,12,3,8,alarm clock bakelike green,12,2010-12-01 08:45:00,3.75,12583,France,45.0,VIP
29,29,536370,21724,2010,12,3,8,panda and bunnies sticker sheet,12,2010-12-01 08:45:00,0.85,12583,France,10.2,VIP
30,30,536370,21883,2010,12,3,8,stars gift tape,24,2010-12-01 08:45:00,0.65,12583,France,15.6,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397883,541868,581584,85038,2011,12,5,12,6 chocolate love heart t-lights,48,2011-12-09 12:25:00,1.85,13777,United Kingdom,88.8,VIP
397905,541890,581586,22061,2011,12,5,12,large cake stand hanging strawbery,8,2011-12-09 12:49:00,2.95,13113,United Kingdom,23.6,VIP
397906,541891,581586,23275,2011,12,5,12,set of 3 hanging owls ollie beak,24,2011-12-09 12:49:00,1.25,13113,United Kingdom,30.0,VIP
397907,541892,581586,21217,2011,12,5,12,red retrospot round cake tins,24,2011-12-09 12:49:00,8.95,13113,United Kingdom,214.8,VIP


In [9]:
# group by country and count unique customers ids
vip_country_counts = merged_orders.groupby("Country")["CustomerID"].nunique()
vip_country_counts 

Country
Australia                  9
Austria                   11
Bahrain                    2
Belgium                   25
Brazil                     1
Canada                     4
Channel Islands            9
Cyprus                     8
Czech Republic             1
Denmark                    9
EIRE                       3
European Community         1
Finland                   12
France                    87
Germany                   94
Greece                     4
Iceland                    1
Israel                     3
Italy                     14
Japan                      8
Lebanon                    1
Lithuania                  1
Malta                      2
Netherlands                9
Norway                    10
Poland                     6
Portugal                  19
RSA                        1
Saudi Arabia               1
Singapore                  1
Spain                     30
Sweden                     8
Switzerland               21
USA                        4
United

In [10]:
#sort thre results 
sorted_vip_countries = vip_country_counts.sort_values(ascending = False)
sorted_vip_countries
# ok there is a clear winner,people in united kingdom really like spending 

Country
United Kingdom          3921
Germany                   94
France                    87
Spain                     30
Belgium                   25
Switzerland               21
Portugal                  19
Italy                     14
Finland                   12
Austria                   11
Norway                    10
Netherlands                9
Australia                  9
Denmark                    9
Channel Islands            9
Cyprus                     8
Sweden                     8
Japan                      8
Poland                     6
USA                        4
Canada                     4
Unspecified                4
Greece                     4
EIRE                       3
Israel                     3
Malta                      2
United Arab Emirates       2
Bahrain                    2
Czech Republic             1
Lithuania                  1
Lebanon                    1
RSA                        1
Saudi Arabia               1
Singapore                  1
Icelan

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [22]:
# Filter for VIP and Preferred customers
vip_preferred_customers = merged_orders[merged_orders['CustomerLabel'].isin(['VIP', 'Preferred'])]

# Group by 'Country' and count unique 'CustomerID's
vip_preferred_country_counts = vip_preferred_customers.groupby('Country')['CustomerID'].nunique()

# Sort the results in descending order
sorted_vip_preferred_countries = vip_preferred_country_counts.sort_values(ascending=False)

sorted_vip_preferred_countries

Country
United Kingdom     737
Germany             33
France              27
Belgium              9
Spain                9
Switzerland          8
Portugal             6
Norway               6
Cyprus               4
Japan                4
Italy                4
Austria              3
Australia            3
Finland              3
EIRE                 3
Denmark              3
Israel               2
Channel Islands      2
Sweden               2
Greece               1
Netherlands          1
Poland               1
Singapore            1
Canada               1
Iceland              1
Name: CustomerID, dtype: int64